In [6]:
import statsmodels.formula.api as sm

In [7]:
sm.ols

<bound method Model.from_formula of <class 'statsmodels.regression.linear_model.OLS'>>

In [8]:
import pandas as pd
import numpy as np
import random
import sklearn as sk
#from pandas.stats.api import ols
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import SelectFromModel

In [10]:
#Read in data and preprocessing 

df = pd.read_csv('prediction_resources/resources/data clean/prediction.csv',index_col='code')
#delete missing data
filtered_df = df[~np.isnan(df['2012_CPI']) & (~np.isnan(df['2012_revenue']))]
#normalization 
norm_factor = ['2012_CPI','2012_GDP','2012_POP','2012_revenue','2012_rev/pop','2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','2008/TOTAL','2012/TOTAL']
normed_df = (filtered_df[norm_factor] - filtered_df[norm_factor].mean())/filtered_df[norm_factor].std()
normed_df[['2008','2012','2016','2008_s','2012_s','2016_s','2008_b','2012_b','2016_b','2008_r','2012_r','2016_r','HOST']]=filtered_df[['2008','2012','2016','2008_s','2012_s','2016_s','2008_b','2012_b','2016_b','2008_r','2012_r','2016_r','HOST']]

In [12]:
df

2008  2012  2016  2012_CPI      2012_GDP    2012_POP  2012_revenue  \
code                                                                       
AFG      0     0     0     7.218  2.053654e+10    29726803         7.989   
ARG     20     1     3    10.030  5.490000e+11    42095224           NaN   
ARM      0     0     1     2.555  1.061932e+10     2978339        22.860   
AUS     31     8     8     1.763  1.540000e+12    22728254        26.596   
AUT      0     0     0     2.486  4.070000e+11     8429991        43.565   
AZE      1     2     1     1.014  6.873091e+10     9295784        49.302   
BEL      1     0     2     2.840  4.980000e+11    11128246        40.717   
BLR      8     2     1    59.220  6.361545e+10     9464000        38.892   
BRA     14     3     7     5.402  2.460000e+12   202401584        25.837   
CAN     11     1     4     1.516  1.820000e+12    34751476        17.720   
CHN     74    38    26     2.625  8.460000e+12  1350695000        11.164   
COL      0     1     3     3.177  3.700000e+11    46881018        21.740   
CUB      2     5     5       NaN  7.313905e+10    11342631           NaN   
CZE      3     4     1     3.299  2.060000e+11    10510785        28.874   
DOM      1     1     0     3.695  6.059511e+10    10155036        15.557   
ESP      7     3     7     2.446  1.340000e+12    46773055        14.946   
EST      1     0     0     3.935  2.313527e+10     1322696         1.977   
ETH      4     3     1    22.770  4.331072e+10    92191211        10.143   
FIN      1     0     0     2.808  2.570000e+11     5413971        38.940   
FRA     25    11    10     1.956  2.680000e+12    65659790        42.378   
GBR     31    29    27     2.822  2.630000e+12    63700300        37.547   
GEO      3     1     2    -0.944  1.584647e+10     3825000        25.691   
HUN     16     8     8     5.668  1.270000e+11     9920362        40.981   
IND      1     0     0     9.312  1.820000e+12  1263589639        12.497   
IRL      0     1     0     1.693  2.250000e+11     4586897        31.872   
ISR      0     0     0     1.708  2.600000e+11     7910500        34.065   
ITA      8     8     8     3.041  2.070000e+12    59539717        36.590   
JAM      9     4     6     6.898  1.474642e+10     2707805        31.702   
JPN     23     7    12    -0.033  5.960000e+12   127561489        12.931   
KAZ      2     7     3     5.114  2.160000e+11    16791425           NaN   
KEN      6     2     6     9.378  5.041016e+10    42542978        16.473   
LTU      0     2     0     3.087  4.285220e+10     2987773         9.170   
MAR      0     0     0     1.279  9.826631e+10    32984190        34.943   
MDA      0     0     0     4.640  7.284687e+09     3559519        34.361   
MEX      2     1     0     4.112  1.190000e+12   122070963           NaN   
NOR     16     2     0     0.709  5.100000e+11     5018573        51.701   
NZL      4     6     4     0.883  1.770000e+11     4408100        36.268   
POL      6     2     2     3.557  5.000000e+11    38063164        32.662   
ROU      5     2     1     3.334  1.720000e+11    20058035        31.708   
SRB      0     1     2     7.330  4.074231e+10     7199077        38.209   
SWE      0     1     2     0.888  5.440000e+11     9519374        33.383   
THA      2     0     2     3.020  3.970000e+11    67164130        18.819   
TJK      0     0     1     5.831  7.633036e+09     7930929           NaN   
TUN      1     2     0     5.138  4.504418e+10    10777500        29.179   
TUR      1     1     1     8.892  7.890000e+11    74849187        31.166   
UKR     10     6     2     0.556  1.760000e+11    45593300        35.822   
USA    125    46    46     2.069  1.620000e+13   314102623        17.771   
UZB      1     1     4       NaN  5.118344e+10    29774500           NaN   

      2012_rev/pop  2008_CPI      2008_GDP  ...     2012Z  2008_b  2012_b  \
code                                        ...                             
AFG   2.690000e-07    30.555  1.019053e+10  ... -0.

In [11]:
#split data into training and test sets
#using original data
rows = random.sample(filtered_df.index, 8)
test = filtered_df[filtered_df.index.isin(rows)]
train = filtered_df.drop(rows)
y = train[['2016','2016_s','2016_b','2016_r']]
X = train.drop(['2016','2016_s','2016_b','2016_r'], axis=1)

#using normalized data
rows2 = random.sample(normed_df.index, 8)
test2 = normed_df[normed_df.index.isin(rows)]
train2 = normed_df.drop(rows)
y2 = train2[['2016','2016_s','2016_b','2016_r']]
X2 = train2.drop(['2016','2016_s','2016_b','2016_r'], axis=1)


#sklearn creat ridge linear model 
reg = linear_model.Lasso(alpha = .3,fit_intercept = True)

TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [464]:
X

2008  2012  2012_CPI      2012_GDP    2012_POP  2012_revenue  \
code                                                                 
AFG      0     0     7.218  2.053654e+10    29726803         7.989   
ARM      0     0     2.555  1.061932e+10     2978339        22.860   
AUS     31     8     1.763  1.540000e+12    22728254        26.596   
AUT      0     0     2.486  4.070000e+11     8429991        43.565   
AZE      1     2     1.014  6.873091e+10     9295784        49.302   
BEL      1     0     2.840  4.980000e+11    11128246        40.717   
BLR      8     2    59.220  6.361545e+10     9464000        38.892   
BRA     14     3     5.402  2.460000e+12   202401584        25.837   
CAN     11     1     1.516  1.820000e+12    34751476        17.720   
CHN     74    38     2.625  8.460000e+12  1350695000        11.164   
COL      0     1     3.177  3.700000e+11    46881018        21.740   
CZE      3     4     3.299  2.060000e+11    10510785        28.874   
DOM      1     1     3.695  6.059511e+10    10155036        15.557   
ESP      7     3     2.446  1.340000e+12    46773055        14.946   
EST      1     0     3.935  2.313527e+10     1322696         1.977   
FIN      1     0     2.808  2.570000e+11     5413971        38.940   
FRA     25    11     1.956  2.680000e+12    65659790        42.378   
GBR     31    29     2.822  2.630000e+12    63700300        37.547   
HUN     16     8     5.668  1.270000e+11     9920362        40.981   
IND      1     0     9.312  1.820000e+12  1263589639        12.497   
ITA      8     8     3.041  2.070000e+12    59539717        36.590   
JAM      9     4     6.898  1.474642e+10     2707805        31.702   
JPN     23     7    -0.033  5.960000e+12   127561489        12.931   
KEN      6     2     9.378  5.041016e+10    42542978        16.473   
MAR      0     0     1.279  9.826631e+10    32984190        34.943   
MDA      0     0     4.640  7.284687e+09     3559519        34.361   
NZL      4     6     0.883  1.770000e+11     4408100        36.268   
POL      6     2     3.557  5.000000e+11    38063164        32.662   
ROU      5     2     3.334  1.720000e+11    20058035        31.708   
SRB      0     1     7.330  4.074231e+10     7199077        38.209   
TUN      1     2     5.138  4.504418e+10    10777500        29.179   
TUR      1     1     8.892  7.890000e+11    74849187        31.166   
UKR     10     6     0.556  1.760000e+11    45593300        35.822   
USA    125    46     2.069  1.620000e+13   314102623        17.771   

      2012_rev/pop  2008_CPI      2008_GDP    2008_POP   ...    2012/TOTAL  \
code                                                     ...                 
AFG   2.690000e-07    30.555  1.019053e+10    26528741   ...      0.000000   
ARM   7.680000e-06     8.950  1.166204e+10     2975029   ...      0.000000   
AUS   1.170000e-06     4.353  1.050000e+12    21249200   ...      0.026667   
AUT   5.170000e-06     3.216  4.280000e+11     8321496   ...      0.000000   
AZE   5.300000e-06    20.792  4.885248e+10     8763400   ...      0.006667   
BEL   3.660000e-06     4.489  5.190000e+11    10709973   ...      0.000000   
BLR   4.110000e-06    14.838  6.075218e+10     9528000   ...      0.006667   
BRA   1.280000e-07     5.663  1.700000e+12   194769696   ...      0.010000   
CAN   5.100000e-07     2.370  1.550000e+12    33245773   ...      0.003333   
CHN   8.270000e-09     5.864  4.560000e+12  1324655000   ...      0.126667   
COL   4.640000e-07     6.997  2.440000e+11    44901660   ...      0.003333   
CZE   2.750000e-06     6.351  2.350000e+11    10384603   ...      0.013333   
DOM   1.530000e-06    10.645  4.815299e+10     9636491   ...      0.003333   
ESP   3.200000e-07     4.076  1.630000e+12    45954106   ...      0.010000   
EST   1.490000e-06    10.366  2.419404e+10     1337090   ...      0.000000   
FIN   7.190000e-06     4.066  2.840000e+11     5313399   ...      0.000000   
FRA   6.450000e-07     2.814  2.920000e+12    64374990   ...      0.036667   
GBR

In [491]:
#model1 y = 2008z + 2012z

# # res1 = ols(y=y['2016'], x=X[['2008','2012']])
# res1 = reg.fit(X[['2008Z','2012Z']],y[['2016']])
# pred1 = res1.predict(test[['2008Z','2012Z']])
# error1 = np.mean((pred1-test['2016'])**2)
# scores1 = res1.score(test[['2008Z','2012Z']],test['2016'])
# print error1, scores1, res1.coef_
# #model1 result: 1.09719153004 0.917193092072 
# # 2008 and 2012 data sufficient

res12 = reg.fit(X2[['2008','2012']],y2[['2016']])
error12 = np.mean((res12.predict(test2[['2008','2012']])-test2['2016'])**2)
scores12 = res1.score(test[['2008','2012']],test['2016'])
print error12, scores12, res12.coef_, res12.intercept_

15.1075196068 0.898061916584 [ 0.08508888  0.77695321] [ 0.39845438]


In [492]:
#model3 y = all
var3 = X2[['2012_CPI','2012_GDP','2012_POP','2012_revenue','2012_rev/pop','2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','2008/TOTAL','2012/TOTAL','HOST','2008','2012']]
testvar3 = test2[['2012_CPI','2012_GDP','2012_POP','2012_revenue','2012_rev/pop','2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','2008/TOTAL','2012/TOTAL','HOST','2008','2012']]
temp = sm.ols(y=y2['2016'], x=var3)

res3= reg.fit(var3,y2['2016'])
error3 = np.mean((res3.predict(testvar3)-test2['2016'])**2)
scores3 = res3.score(testvar3,test2['2016'])
print error3, scores3, res3.coef_, temp
#model3 result:
# R-squared:         0.9759
# Adj R-squared:     0.9602

# F-stat (15, 20):    62.2111, p-value:     0.0000

# Degrees of Freedom: model 13, resid 20

# -----------------------Summary of Estimated Coefficients------------------------
#       Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
# --------------------------------------------------------------------------------
#       2012_CPI     0.1454     0.3827       0.38     0.7080    -0.6047     0.8955
#       2012_GDP    -2.5286     6.9049      -0.37     0.7181   -16.0622    11.0050
#       2012_POP     4.4703    47.3345       0.09     0.9257   -88.3053    97.2459
#   2012_revenue    -0.5569     0.3098      -1.80     0.0874    -1.1641     0.0503 
#   2012_rev/pop    -1.9802     9.8145      -0.20     0.8421   -21.2167    17.2563
# --------------------------------------------------------------------------------
#       2008_CPI    -0.1100     0.5202      -0.21     0.8346    -1.1296     0.9095
#       2008_GDP     5.2200     5.9379       0.88     0.3898    -6.4182    16.8582
#       2008_POP    -4.9183    48.3909      -0.10     0.9201   -99.7646    89.9279
#   2008_revenue     0.2135     0.3098       0.69     0.4987    -0.3938     0.8207
#   2008_rev/pop     1.8434     9.9665       0.18     0.8551   -17.6910    21.3778
# --------------------------------------------------------------------------------
#     2008/TOTAL 80796808.0732     0.2186 369567367.95     0.0000 80796807.6447 80796808.5017
#     2012/TOTAL -23897483.7194     0.2297 -104027887.23     0.0000 -23897484.1697 -23897483.2692
#           HOST    -0.0000     0.0000       -inf     0.0000    -0.0000    -0.0000
#           2008 -3593851.0503     0.0776 -46316418.12     0.0000 -3593851.2024 -3593850.8982
#           2012 2449032.9652     0.1335 18351058.71     0.0000 2449032.7036 2449033.2268
# --------------------------------------------------------------------------------
#      intercept 26291439.5893     0.4563 57615753.38     0.0000 26291438.6949 26291440.4837
# ---------------------------------End of Summary---------------------------------

#useful indicators: 2008/TOTAL, 2012/TOTAL, HOST, 2008, 2012

16.4125753938 0.767815025375 [-0.          0.         -0.         -0.17772193 -0.         -0.1965602   0.
 -0.         -0.094299   -0.07054402  0.          0.          0.
  0.07612379  0.79135313] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2012_CPI> + <2012_GDP> + <2012_POP> + <2012_revenue>
             + <2012_rev/pop> + <2008_CPI> + <2008_GDP> + <2008_POP> + <2008_revenue>
             + <2008_rev/pop> + <2008/TOTAL> + <2012/TOTAL> + <HOST> + <2008> + <2012>
             + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   15

R-squared:         0.9745
Adj R-squared:     0.9558

Rmse:              1.8866

F-stat (15, 19):    51.9145, p-value:     0.0000

Degrees of Freedom: model 14, resid 19

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
-------------------------------------------------

In [493]:
#model3 refinement: select feature
model3 = SelectFromModel(res3, prefit=True)
var3_new = model3.transform(var3)
testvar3_new = model3.transform(testvar3)
#final model
model3_new = reg.fit(var3_new,y2['2016'])

#prediction
test['pred3_new'] = model3_new.predict(testvar3_new)
#model evaluation
error3_new = np.mean((test['pred3_new']-test['2016'])**2)
scores3_new = model3_new.score(testvar3_new, test['2016'])
print error3_new, scores3_new, model3_new.coef_
#model3_new result: '2012_CPI','2012_POP','2012_revenue','2008_GDP','2008_revenue','2008','2012' 

16.4125355535 0.767815588986 [-0.2391903  -0.1965589  -0.0328278  -0.07054548  0.07612463  0.79135125]


/Users/huchenchen/anaconda/envs/PYTHON2.7/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [494]:
#model4: a combination of model3 and model3_new
#model3: 2008/TOTAL, 2012/TOTAL, HOST, 2008, 2012
#model3_new: '2012_CPI','2012_POP','2012_revenue','2008_GDP','2008_revenue','2008','2012' 
#Since 2016 economic data is missing, trying to drop 2012 economics data from model4
#y = '2008_GDP','2008_revenue','2008','2012','HOST','2008/TOTAL','2012/TOTAL','2008_CPI','2008_POP','2008_rev/pop'
var4 = X2[['2008_GDP','2008_revenue','2008','2012','HOST','2008/TOTAL','2012/TOTAL','2008_CPI','2008_POP','2008_rev/pop']]
testvar4 = test2[['2008_GDP','2008_revenue','2008','2012','HOST','2008/TOTAL','2012/TOTAL','2008_CPI','2008_POP','2008_rev/pop']]
res4 = reg.fit(var4, y2['2016'])

temp = ols(y=y2['2016'], x=var4)
error4 = np.mean((res4.predict(testvar4)-test2['2016'])**2)
scores4 = res4.score(testvar4, test2['2016'])
print error4, scores4,res4.coef_,temp
#2012_revenue','2008_CPI','2008_GDP','2008_POP','2008','2012'

16.4131557454 0.767806815273 [ 0.         -0.2720572   0.07611059  0.79138214  0.          0.          0.
 -0.19659712 -0.         -0.07052961] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2008_GDP> + <2008_revenue> + <2008> + <2012> + <HOST>
             + <2008/TOTAL> + <2012/TOTAL> + <2008_CPI> + <2008_POP> + <2008_rev/pop>
             + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   11

R-squared:         0.9739
Adj R-squared:     0.9625

Rmse:              1.7364

F-stat (10, 23):    85.7423, p-value:     0.0000

Degrees of Freedom: model 10, resid 23

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      2008_GDP     2.1526     1.9448       1.11     0.2798    -1.6593     5.9645
  2008_revenue    -0.3950 

In [531]:
#model5 
var5 = X2[['2008/TOTAL','2012/TOTAL','HOST','2008','2012']]
testvar5 = test2[['2008/TOTAL','2012/TOTAL','HOST','2008','2012']]
res5 = reg.fit(var5,y2['2016'])
temp = ols(y=y2['2016'],x=var5)
error5 = np.mean(((res5.predict(testvar5)-test2['2016']))**2)
scores5 = res5.score(testvar5,test2['2016'])
print error5, scores5, res5.coef_,temp
#model5 result: 5.4982199398 0.970014071009
#'2008_CPI','2008_GDP','2008_POP','2008_revenue','2008','2012'

15.1080897376 0.786269287532 [ 0.          0.          0.          0.08507278  0.77698943] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2008/TOTAL> + <2012/TOTAL> + <HOST> + <2008> + <2012>
             + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   6

R-squared:         0.9640
Adj R-squared:     0.9575

Rmse:              1.8485

F-stat (5, 28):   149.7870, p-value:     0.0000

Degrees of Freedom: model 5, resid 28

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
    2008/TOTAL 35591480.4573 8689492.4739       4.10     0.0003 18560075.2085 52622885.7062
    2012/TOTAL -72271411.6316        nan        nan        nan        nan        nan
          HOST     3.6886     1.9046       1.94     0.0629    -0.0443   

In [530]:
#model6 y = '2008_CPI','2008_GDP','2008_POP','2008_revenue','2008','2012'
#original data
var6 = X[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008','2012']]
testvar6 = test[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008','2012']]
res6 = reg.fit(var6,y['2016'])
error6 = np.mean(((res6.predict(testvar6)-test['2016']))**2)
scores6 = res6.score(testvar6,test['2016'])
print error6, scores6, res6.coef_

#normalized data
var62 = X2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008','2012']]
testvar62 = test2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008','2012']]
res62 = reg.fit(var62,y2['2016'])
temp = ols(y=y['2016'],x=var62)
error62 = np.mean(((res62.predict(testvar62)-test2['2016']))**2)
scores62 = res62.score(testvar62,test2['2016'])
print error62, scores62, res62.coef_,temp

7.68997680591 0.89121164554 [ -3.95718731e-02   7.04509943e-13  -1.42682188e-09  -4.12588606e-02
  -0.00000000e+00   7.86712328e-01]
16.5551326668 0.765798300028 [-0.20256619  0.         -0.         -0.31596973  0.07517971  0.79503542] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2008_CPI> + <2008_GDP> + <2008_POP> + <2008_revenue> + <2008>
             + <2012> + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   7

R-squared:         0.9679
Adj R-squared:     0.9607

Rmse:              1.7777

F-stat (6, 27):   135.4977, p-value:     0.0000

Degrees of Freedom: model 6, resid 27

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      2008_CPI    -0.4055     0.3263      -1.24     0.2247    -1.0451     0.2341
      

In [528]:
#we use model6 as final data

var_2020 = normed_df[['2012_CPI','2012_GDP','2012_POP','2012_revenue','2012','2016']]
predg = res62.predict(var_2020)
predg[predg <0]=0
normed_df['2020_G'] = np.round(predg)

In [501]:
#model for silver
#model 1 y = all
var1s = X2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','HOST','2008_s','2012_s','2008','2012']]
testvar1s = test2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','HOST','2008_s','2012_s','2008','2012']]
temp = ols(y=y2['2016_s'],x=var1s)


res1s= reg.fit(var1s,y2['2016_s'])
error1s = np.mean((res1s.predict(testvar1s)-test2['2016_s'])**2)
scores1s = res1s.score(testvar1s,test2['2016_s'])
print error1s, scores1s, res1s.coef_,temp
#result: '2008_s','2012_s','2008','2012','2008_revenue'


34.4904118749 0.311697424386 [ 0.          0.          0.          0.03775057 -0.          0.
  0.03177112  0.26338721 -0.0433145   0.66332562] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2008_CPI> + <2008_GDP> + <2008_POP> + <2008_revenue>
             + <2008_rev/pop> + <HOST> + <2008_s> + <2012_s> + <2008> + <2012> + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   11

R-squared:         0.9519
Adj R-squared:     0.9310

Rmse:              1.9316

F-stat (10, 23):    45.5576, p-value:     0.0000

Degrees of Freedom: model 10, resid 23

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      2008_CPI     0.5481     0.3810       1.44     0.1637    -0.1986     1.2948
      2008_GDP     1.8352     1.1290       1.63

In [515]:
#model2 for sliver
# y = '2008_s','2012_s','2008','2012','2008_revenue'

var2s = X2[['2008_s','2012_s','2008','2012','2008_revenue']]
testvar2s = test2[['2008_s','2012_s','2008','2012','2008_revenue']]
temp = ols(y=y2['2016_s'],x=var2s)


res2s= reg.fit(var2s,y2['2016_s'])
error2s = np.mean((res2s.predict(testvar2s)-test2['2016_s'])**2)
scores2s = res2s.score(testvar2s,test2['2016_s'])
print error2s, scores2s, res2s.coef_,temp

34.4895843994 0.311713937774 [ 0.0318024   0.26339686 -0.04336158  0.66336615  0.03780755] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2008_s> + <2012_s> + <2008> + <2012> + <2008_revenue>
             + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   6

R-squared:         0.9422
Adj R-squared:     0.9318

Rmse:              1.9206

F-stat (5, 28):    91.2090, p-value:     0.0000

Degrees of Freedom: model 5, resid 28

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
        2008_s     0.0406     0.0416       0.97     0.3380    -0.0410     0.1221
        2012_s     0.3014     0.1343       2.24     0.0330     0.0381     0.5647
          2008    -0.0631     0.0531      -1.19     0.2444    -0.1672     0.0409
      

In [524]:
#use model2 as final model to predict silvers
var_2020s = normed_df[['2012_s','2016_s','2012','2016','2012_revenue']]
preds = res2s.predict(var_2020s)
preds[preds <0]=0
normed_df['2020_S'] = np.round(preds)

In [529]:
normed_df

2012_CPI  2012_GDP  2012_POP  2012_revenue  2012_rev/pop  2008_CPI  \
code                                                                       
AFG   0.222004 -0.432813 -0.249833     -1.652369     -0.842356  2.377871   
ARM  -0.274860 -0.436257 -0.345490     -0.422535      1.446848 -0.047554   
AUS  -0.359251  0.094928 -0.274861     -0.113567     -0.564044 -0.563623   
AUT  -0.282212 -0.298586 -0.325994      1.289772      0.671527 -0.691265   
AZE  -0.439060 -0.416074 -0.322898      1.764223      0.711683  1.281855   
BEL  -0.244492 -0.266980 -0.316345      1.054242      0.205099 -0.548355   
BLR   5.763052 -0.417851 -0.322297      0.903314      0.344101  0.613446   
BRA   0.028501  0.414463  0.367680     -0.176336     -0.885910 -0.416560   
CAN  -0.385570  0.192178 -0.231864     -0.847614     -0.767913 -0.786239   
CHN  -0.267401  2.498388  4.474165     -1.389796     -0.922893 -0.393995   
COL  -0.208583 -0.311437 -0.188487     -0.515159     -0.782122 -0.266802   
CZE  -0.195583 -0.368398 -0.318553      0.074824     -0.075993 -0.339323   
DOM  -0.153388 -0.418900 -0.319825     -1.026494     -0.452842  0.142731   
ESP  -0.286474  0.025464 -0.188873     -1.077024     -0.826602 -0.594720   
EST  -0.127815 -0.431910 -0.351411     -2.149563     -0.465198  0.111410   
ETH   1.879140 -0.424903 -0.026450     -1.474233     -0.891470  3.931131   
FIN  -0.247902 -0.350684 -0.336780      0.907284      1.295490 -0.595842   
FRA  -0.338686  0.490874 -0.121331      1.191607     -0.726212 -0.736395   
GBR  -0.246410  0.473508 -0.128339      0.792083     -0.743510 -0.646697   
GEO  -0.647694 -0.434442 -0.342462     -0.188411      1.150311  0.070209   
HUN   0.056844 -0.395836 -0.320664      1.076075      0.350279 -0.371318   
IND   0.445129  0.192178  4.162662     -1.279557     -0.922393 -0.114687   
IRL  -0.366710 -0.361799 -0.339738      0.322759      1.221356 -0.597190   
ISR  -0.365112 -0.349642 -0.327852      0.504121      0.405879 -0.536231   
ITA  -0.223074  0.279008 -0.143218      0.712939     -0.735479 -0.673416   
JAM   0.187906 -0.434824 -0.346458      0.308700      2.688596  1.419826   
JPN  -0.550623  1.630086  0.100039     -1.243665     -0.894250 -0.898164   
KEN   0.452162 -0.422437 -0.204001     -0.950741     -0.805906  1.893460   
LTU  -0.218173 -0.425062 -0.345457     -1.554700      0.022852  0.174389   
MAR  -0.410823 -0.405816 -0.238184      0.576731     -0.598022 -0.636144   
MDA  -0.052694 -0.437416 -0.343412      0.528600      2.055366  0.395545   
NOR  -0.471560 -0.262812 -0.338194      1.962621      2.256146 -0.629521   
NZL  -0.453019 -0.378470 -0.340377      0.686309      1.616738 -0.607854   
POL  -0.168092 -0.266285 -0.220021      0.388092     -0.660418 -0.564072   
ROU  -0.191854 -0.380206 -0.284411      0.309196     -0.437398 -0.171267   
SRB   0.233938 -0.425795 -0.330396      0.846830      0.714772  0.340986   
SWE  -0.452486 -0.251003 -0.322098      0.447719      0.158765 -0.666455   
THA  -0.225312 -0.302059 -0.115951     -0.756726     -0.838958 -0.438451   
TUN   0.000370 -0.424301 -0.317599      0.100048     -0.088349 -0.499858   
TUR   0.400376 -0.165910 -0.088468      0.264373     -0.796949  0.120166   
UKR  -0.487862 -0.378817 -0.193092      0.649425     -0.682658  1.780300   
USA  -0.326645  5.186650  0.767141     -0.843396     -0.907965 -0.621326   

      2008_GDP  2008_POP  2008_revenue  2008_rev/pop   ...    2016_s  2008_b  \
code                                                   ...                     
AFG  -0.435818 -0.257886     -1.652369     -0.833227   ...         0       1   
ARM  -0.435218 -0.345137     -0.422535      1.408580   ...         3       6   
AUS  -0.011662 -0.277443     -0.113567     -0.544912   ...        11      76   
AUT  -0.265387 -0.325332      1.289772      0.667286   ...         0       2   
AZE  -0.420047 -0.323695      1.764223      0.785771   ...         7       4   
BEL  -0.228266 -0.316484      1.054242      0.229801   ...         2       0   
BLR  -0.415193 -0.320

In [532]:
#model for bronze
#model 1 y = all
var1b = X2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','HOST','2008_s','2012_s','2008','2012','2008_b','2012_b']]
testvar1b = test2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','HOST','2008_s','2012_s','2008','2012','2008_b','2012_b']]
temp = ols(y=y2['2016_b'],x=var1b)


res1b= reg.fit(var1b,y2['2016_b'])
error1b = np.mean((res1b.predict(testvar1b)-test2['2016_b'])**2)
scores1b = res1b.score(testvar1b,test2['2016_b'])
print error1b, scores1b, res1b.coef_,temp
#result: 
#'2012_b','2008_GDP','2008_b','2008_POP','2012'
# R-squared:         0.9455
# Adj R-squared:     0.9144

# Rmse:              2.1945

# F-stat (12, 21):    30.3846, p-value:     0.0000

# Degrees of Freedom: model 12, resid 21

# -----------------------Summary of Estimated Coefficients------------------------
#       Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
# --------------------------------------------------------------------------------
#       2008_CPI     0.1181     0.4374       0.27     0.7898    -0.7392     0.9755
#       2008_GDP     3.0761     1.4787       2.08     0.0499     0.1780     5.9743
#       2008_POP    -0.9585     0.5599      -1.71     0.1017    -2.0560     0.1390
#   2008_revenue     0.2822     0.5282       0.53     0.5988    -0.7532     1.3176
#   2008_rev/pop    -0.1461     0.5106      -0.29     0.7776    -1.1468     0.8546
# --------------------------------------------------------------------------------
#           HOST    -3.1145     2.8078      -1.11     0.2799    -8.6178     2.3889
#         2008_s     0.0147     0.0605       0.24     0.8097    -0.1038     0.1333
#         2012_s     0.0628     0.2818       0.22     0.8257    -0.4896     0.6152
#           2008     0.0568     0.0979       0.58     0.5679    -0.1351     0.2486
#           2012    -0.2210     0.1479      -1.49     0.1500    -0.5108     0.0689
# --------------------------------------------------------------------------------
#         2008_b    -0.0534     0.0606      -0.88     0.3883    -0.1720     0.0653
#         2012_b     0.8498     0.2020       4.21     0.0004     0.4539     1.2458
#      intercept     1.4990     0.9756       1.54     0.1394    -0.4133     3.4113
# ---------------------------------End of Summary---------------------------------

3.47336981232 0.948625914493 [-0.          0.         -0.05916173  0.         -0.         -0.          0.0163858
  0.15464815  0.17861211 -0.17233461 -0.09161958  0.83645246] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2008_CPI> + <2008_GDP> + <2008_POP> + <2008_revenue>
             + <2008_rev/pop> + <HOST> + <2008_s> + <2012_s> + <2008> + <2012> + <2008_b>
             + <2012_b> + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   13

R-squared:         0.9455
Adj R-squared:     0.9144

Rmse:              2.1945

F-stat (12, 21):    30.3846, p-value:     0.0000

Degrees of Freedom: model 12, resid 21

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
      2008_CPI     0.1181     0.4374       0.27     0.7898    

In [537]:
#model 2 for bronze
#y = '2012_b','2008_GDP','2008_b','2008_POP','2012'
var2b = X2[['2012_b','2008_GDP','2008_POP','2012']]
testvar2b = test2[['2012_b','2008_GDP','2008_POP','2012']]
temp = ols(y=y2['2016_b'],x=var2b)


res2b= reg.fit(var2b,y2['2016_b'])
error2b = np.mean((res2b.predict(testvar2b)-test2['2016_b'])**2)
scores2b = res2b.score(testvar2b,test2['2016_b'])
print error2b, scores2b, res2b.coef_,temp
#result: 0.9289
# R-squared:         0.9375
# Adj R-squared:     0.9289

# Rmse:              2.0005

# F-stat (4, 29):   108.7598, p-value:     0.0000

# Degrees of Freedom: model 4, resid 29

# -----------------------Summary of Estimated Coefficients------------------------
#       Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
# --------------------------------------------------------------------------------
#         2012_b     0.7753     0.1225       6.33     0.0000     0.5353     1.0154
#       2008_GDP     3.5954     0.7391       4.86     0.0000     2.1467     5.0441
#       2008_POP    -1.0739     0.4821      -2.23     0.0338    -2.0188    -0.1290
#           2012    -0.1125     0.0928      -1.21     0.2354    -0.2945     0.0695
#      intercept     1.7690     0.6912       2.56     0.0160     0.4142     3.1238
# ---------------------------------End of Summary---------------------------------

6.94112158023 0.897334924628 [ 0.82615035  2.0949512  -0.23255157  0.        ] 
-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <2012_b> + <2008_GDP> + <2008_POP> + <2012> + <intercept>

Number of Observations:         34
Number of Degrees of Freedom:   5

R-squared:         0.9375
Adj R-squared:     0.9289

Rmse:              2.0005

F-stat (4, 29):   108.7598, p-value:     0.0000

Degrees of Freedom: model 4, resid 29

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
        2012_b     0.7753     0.1225       6.33     0.0000     0.5353     1.0154
      2008_GDP     3.5954     0.7391       4.86     0.0000     2.1467     5.0441
      2008_POP    -1.0739     0.4821      -2.23     0.0338    -2.0188    -0.1290
          2012    -0.1125     0.0928      

In [538]:
#use model2 as the final model to predict bronze numbers
var_2020b = normed_df[['2016_b','2012_GDP','2012_POP','2016']]
predb = res2b.predict(var_2020b)
predb[predb <0]=0
normed_df['2020_B'] = np.round(predb)

In [659]:
#model2: use past data

#Ranking data value top ranks most, need to assign sample weight
#assume sample weight = (n-rank)/n
#in 2012 79 countries win at least one medal

X2['weight'] = (max(X2['2012_r'])-X2['2012_r'])**2/max(X2['2012_r'])
var1r = X2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','HOST','2008_s','2012_s','2008','2012','2008_b','2012_b','2008_r','2012_r']]
testvar1r = test2[['2008_CPI','2008_GDP','2008_POP','2008_revenue','2008_rev/pop','HOST','2008_s','2012_s','2008','2012','2008_b','2012_b','2008_r','2012_r']]

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
temp = sm.WLS(y2['2016_r'],var1r,weights=X2['weight']).fit()


# res1r= linear_model.LinearRegression().fit(var1r,y2['2016_r'],sample_weight=X2['weight'])
res1r= linear_model.LinearRegression().fit(var1r,y2['2016_r'])
error1r = np.mean((res1r.predict(testvar1r)-test2['2016_r'])**2)
scores1r = res1r.score(testvar1r,test2['2016_r'])
print error1r, scores1r, res1r.coef_,temp.summary()
#result '2008_s','2012_b','2008_r','2012_r','2008','2008_GDP'
# Dep. Variable:                 2016_r   R-squared:                       0.902
# Model:                            WLS   Adj. R-squared:                  0.833
# Method:                 Least Squares   F-statistic:                     13.13
# Date:                Mon, 05 Dec 2016   Prob (F-statistic):           3.79e-07
# Time:                        04:23:25   Log-Likelihood:                   -inf
# No. Observations:                  34   AIC:                               inf
# Df Residuals:                      20   BIC:                               inf
# Df Model:                          14                                         
# Covariance Type:            nonrobust                                         
# ================================================================================
#                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
# --------------------------------------------------------------------------------
# 2008_CPI         1.1213      3.040      0.369      0.716        -5.221     7.463
# 2008_GDP        -5.0457      8.805     -0.573      0.573       -23.413    13.322
# 2008_POP         1.2223      5.026      0.243      0.810        -9.262    11.706
# 2008_revenue     1.4198      4.140      0.343      0.735        -7.216    10.056
# 2008_rev/pop     0.5511      3.736      0.148      0.884        -7.242     8.345
# HOST            -1.2816     18.387     -0.070      0.945       -39.637    37.074
# 2008_s          -0.3233      0.381     -0.848      0.407        -1.119     0.472
# 2012_s           0.3099      1.945      0.159      0.875        -3.747     4.367
# 2008             0.5916      0.727      0.813      0.426        -0.926     2.109
# 2012             0.2513      1.071      0.235      0.817        -1.983     2.485
# 2008_b           0.0770      0.418      0.184      0.856        -0.796     0.950
# 2012_b          -1.1717      1.321     -0.887      0.385        -3.926     1.583
# 2008_r           0.1309      0.276      0.475      0.640        -0.444     0.706
# 2012_r           1.0618      0.324      3.278      0.004         0.386     1.737
# ==============================================================================
# Omnibus:                        0.058   Durbin-Watson:                   2.282
# Prob(Omnibus):                  0.971   Jarque-Bera (JB):                0.124
# Skew:                          -0.079   Prob(JB):                        0.940
# Kurtosis:                       2.751   Cond. No.                         371.
# ==============================================================================

316.772369018 0.512739884703 [-0.45963056  3.1934801   1.88944215 -2.14270471  1.92593462 -5.90673573
 -0.4140639  -1.22694935  0.26865664  0.88137726  0.33163328 -2.24721173
 -0.03100702  0.73861908]                             WLS Regression Results                            
Dep. Variable:                 2016_r   R-squared:                       0.893
Model:                            WLS   Adj. R-squared:                  0.819
Method:                 Least Squares   F-statistic:                     11.98
Date:                Mon, 05 Dec 2016   Prob (F-statistic):           8.23e-07
Time:                        05:00:16   Log-Likelihood:                   -inf
No. Observations:                  34   AIC:                               inf
Df Residuals:                      20   BIC:                               inf
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
         

In [660]:
#model 2
# y ='2008_s','2012_b','2012_r','2008','2008_GDP'
var2r = X2[['2012_r','2008_r','2012']]
testvar2r = test2[['2012_r','2008_r','2012']]
temp = sm.WLS(y2['2016_r'],var2r,weights=X2['weight']).fit()


res2r= linear_model.LinearRegression().fit(var2r,y2['2016_r'])
error2r = np.mean((res2r.predict(testvar2r)-test2['2016_r'])**2)
scores2r = res2r.score(testvar2r,test2['2016_r'])
print error2r, scores2r, res2r.coef_,temp.summary()
#result
# ==============================================================================
# Dep. Variable:                 2016_r   R-squared:                       0.854
# Model:                            WLS   Adj. R-squared:                  0.840
# Method:                 Least Squares   F-statistic:                     60.64
# Date:                Mon, 05 Dec 2016   Prob (F-statistic):           4.52e-13
# Time:                        05:00:25   Log-Likelihood:                   -inf
# No. Observations:                  34   AIC:                               inf
# Df Residuals:                      31   BIC:                               inf
# Df Model:                           3                                         
# Covariance Type:            nonrobust                                         
# ==============================================================================
#                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
# ------------------------------------------------------------------------------
# 2012_r         1.0780      0.229      4.705      0.000         0.611     1.545
# 2008_r         0.1115      0.209      0.534      0.597        -0.315     0.538
# 2012          -0.0170      0.153     -0.111      0.912        -0.328     0.294
# ==============================================================================
# Omnibus:                        2.303   Durbin-Watson:                   2.085
# Prob(Omnibus):                  0.316   Jarque-Bera (JB):                1.489
# Skew:                           0.507   Prob(JB):                        0.475
# Kurtosis:                       3.153   Cond. No.                         5.04
# ==============================================================================

419.814861057 0.354239644587 [ 0.87159392  0.13124059 -0.17339262]                             WLS Regression Results                            
Dep. Variable:                 2016_r   R-squared:                       0.854
Model:                            WLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     60.64
Date:                Mon, 05 Dec 2016   Prob (F-statistic):           4.52e-13
Time:                        05:00:25   Log-Likelihood:                   -inf
No. Observations:                  34   AIC:                               inf
Df Residuals:                      31   BIC:                               inf
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------

In [661]:
#use model2 as another prediction for 2020_rank
var_2020r = normed_df[['2016_r','2012_r','2016']]
predr2 = res2r.predict(var_2020r)
predr2[predr2 <1]=1
normed_df['2020_R2'] = np.round(predr2)

In [663]:
#model for rank prediction
#model 1: use prediction result of gold, silver, medal and sort
df1 = normed_df.sort_values(by=['2020_G', '2020_S','2020_B'], ascending=[0, 0,0])
df1['2020_R']=0
i= 1
for index, rows in predr.iterrows():
    df1.set_value(index,'2020_R',i)
    i=i+1

In [666]:
#output 2020 prediction
df1[['2020_G','2020_S','2020_B','2020_R','2020_R2']].to_csv('resources/data clean/predresult.csv')

In [665]:
df1

2012_CPI  2012_GDP  2012_POP  2012_revenue  2012_rev/pop  2008_CPI  \
code                                                                       
USA  -0.326645  5.186650  0.767141     -0.843396     -0.907965 -0.621326   
GBR  -0.246410  0.473508 -0.128339      0.792083     -0.743510 -0.646697   
CHN  -0.267401  2.498388  4.474165     -1.389796     -0.922893 -0.393995   
JPN  -0.550623  1.630086  0.100039     -1.243665     -0.894250 -0.898164   
FRA  -0.338686  0.490874 -0.121331      1.191607     -0.726212 -0.736395   
AUS  -0.359251  0.094928 -0.274861     -0.113567     -0.564044 -0.563623   
ITA  -0.223074  0.279008 -0.143218      0.712939     -0.735479 -0.673416   
ESP  -0.286474  0.025464 -0.188873     -1.077024     -0.826602 -0.594720   
HUN   0.056844 -0.395836 -0.320664      1.076075      0.350279 -0.371318   
BRA   0.028501  0.414463  0.367680     -0.176336     -0.885910 -0.416560   
KEN   0.452162 -0.422437 -0.204001     -0.950741     -0.805906  1.893460   
JAM   0.187906 -0.434824 -0.346458      0.308700      2.688596  1.419826   
NZL  -0.453019 -0.378470 -0.340377      0.686309      1.616738 -0.607854   
CAN  -0.385570  0.192178 -0.231864     -0.847614     -0.767913 -0.786239   
COL  -0.208583 -0.311437 -0.188487     -0.515159     -0.782122 -0.266802   
SWE  -0.452486 -0.251003 -0.322098      0.447719      0.158765 -0.666455   
POL  -0.168092 -0.266285 -0.220021      0.388092     -0.660418 -0.564072   
UKR  -0.487862 -0.378817 -0.193092      0.649425     -0.682658  1.780300   
BEL  -0.244492 -0.266980 -0.316345      1.054242      0.205099 -0.548355   
SRB   0.233938 -0.425795 -0.330396      0.846830      0.714772  0.340986   
THA  -0.225312 -0.302059 -0.115951     -0.756726     -0.838958 -0.438451   
CZE  -0.195583 -0.368398 -0.318553      0.074824     -0.075993 -0.339323   
ETH   1.879140 -0.424903 -0.026450     -1.474233     -0.891470  3.931131   
GEO  -0.647694 -0.434442 -0.342462     -0.188411      1.150311  0.070209   
AZE  -0.439060 -0.416074 -0.322898      1.764223      0.711683  1.281855   
TUR   0.400376 -0.165910 -0.088468      0.264373     -0.796949  0.120166   
ROU  -0.191854 -0.380206 -0.284411      0.309196     -0.437398 -0.171267   
ARM  -0.274860 -0.436257 -0.345490     -0.422535      1.446848 -0.047554   
LTU  -0.218173 -0.425062 -0.345457     -1.554700      0.022852  0.174389   
TUN   0.000370 -0.424301 -0.317599      0.100048     -0.088349 -0.499858   
DOM  -0.153388 -0.418900 -0.319825     -1.026494     -0.452842  0.142731   
EST  -0.127815 -0.431910 -0.351411     -2.149563     -0.465198  0.111410   
IND   0.445129  0.192178  4.162662     -1.279557     -0.922393 -0.114687   
AFG   0.222004 -0.432813 -0.249833     -1.652369     -0.842356  2.377871   
BLR   5.763052 -0.417851 -0.322297      0.903314      0.344101  0.613446   
NOR  -0.471560 -0.262812 -0.338194      1.962621      2.256146 -0.629521   
ISR  -0.365112 -0.349642 -0.327852      0.504121      0.405879 -0.536231   
AUT  -0.282212 -0.298586 -0.325994      1.289772      0.671527 -0.691265   
FIN  -0.247902 -0.350684 -0.336780      0.907284      1.295490 -0.595842   
MAR  -0.410823 -0.405816 -0.238184      0.576731     -0.598022 -0.636144   
MDA  -0.052694 -0.437416 -0.343412      0.528600      2.055366  0.395545   
IRL  -0.366710 -0.361799 -0.339738      0.322759      1.221356 -0.597190   

      2008_GDP  2008_POP  2008_revenue  2008_rev/pop   ...    2016_b  2008_r  \
code                                                   ...                     
USA   5.556404  0.770308     -0.843396     -0.906931   ...        38       1   
GBR   0.698113 -0.127204      0.792083     -0.740261   ...        17       8   
CHN   1.420126  4.550806     -1.389796     -0.922112   ...        26       2   
JPN   1.538422  0.118231     -1.243665     -0.893989   ...        21      10   
FRA   0.751143 -0.117691      1.191607     -0.724767   ...        14       9   
AUS  -0.011662 -0.277443     -0.113567     -0.544912   ...        10       7   
ITA   0.534947 -0.138